In [10]:
import torch
import pathlib
import utils
import torchaudio
from torchaudio import transforms
import pandas as pd
import models

In [11]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [18]:
model = models.AudioClassifier()
model.to(device)

AudioClassifier(
  (conv1): Conv2d(2, 8, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
  (relu1): ReLU()
  (bn1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (relu2): ReLU()
  (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (relu3): ReLU()
  (bn3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (relu4): ReLU()
  (bn4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (ap): AdaptiveAvgPool2d(output_size=1)
  (lin): Linear(in_features=64, out_features=5, bias=True)
  (conv): Sequential(
    (0): Conv2d(2, 8, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
    (1): ReLU()
    (2): BatchNorm2d(8, eps=1e-05, momen

In [19]:
model_path = pathlib.Path("_test_model.pth")
model.load_state_dict(torch.load(model_path))
model.eval()

AudioClassifier(
  (conv1): Conv2d(2, 8, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
  (relu1): ReLU()
  (bn1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (relu2): ReLU()
  (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (relu3): ReLU()
  (bn3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (relu4): ReLU()
  (bn4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (ap): AdaptiveAvgPool2d(output_size=1)
  (lin): Linear(in_features=64, out_features=5, bias=True)
  (conv): Sequential(
    (0): Conv2d(2, 8, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
    (1): ReLU()
    (2): BatchNorm2d(8, eps=1e-05, momen

In [20]:
in_path = pathlib.Path("data/soundscape_unimodal2.wav")
labels_in = pd.read_csv(in_path.with_suffix(".txt"), sep = "\t", names = ["onset", "offset", "label"])
labels_in

,onset,offset,label
0,0.863569,8.166184,Electric_shaver_toothbrush
1,3.614554,4.077426,Cat
2,4.750588,5.164475,Dog
3,6.362795,6.644396,Dishes


In [24]:
# predict single soundscape

aud = utils.AudioUtil.open(in_path)
reaud = utils.AudioUtil.resample(aud, 44100)
rechan = utils.AudioUtil.rechannel(reaud, 2)
dur_aud = utils.AudioUtil.pad_trunc(rechan, 10000)
sgram = utils.AudioUtil.spectro_gram(dur_aud, n_mels=64, n_fft=1024, hop_len=None)
audio = torch.unsqueeze(sgram, 0).to(device)

pred = model(audio)
# pred[pred>0.5] = 1
# pred[pred<=0.5] = 0

In [25]:
pred

tensor([[2.7292e+02, 1.2788e+02, 1.1954e-03, 1.6866e+02, 1.7658e+03]],
       device='cuda:0', grad_fn=<AddmmBackward>)

In [ ]:
'''
CLASSES
array(['Alarm_bell_ringing', 'Cat', 'Dishes', 'Dog',
       'Electric_shaver_toothbrush'], dtype=object)

'''